In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import networkx as nx
import random
import pickle
import itertools
import math
import urllib.request as urllib
import io
import zipfile

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.linalg import diag
from tensorflow.keras import callbacks
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import normalized_mutual_info_score
from networkx.generators.community import LFR_benchmark_graph
from itertools import count
from scipy.spatial import distance_matrix
from sklearn.decomposition import NMF
from rdyn import RDyn

from helpers import *
from Autoencoder import *

In [2]:
graphlist = pickle.load(open('graphs/thiers_day_19.p', 'rb'))

In [3]:
t1 = graphlist[0]

In [7]:
t1_A = create_adjacency_matrix(t1)
t1_B = tf.convert_to_tensor(nx.modularity_matrix(t1).astype('float32'))

In [8]:
model_t1 = Autoencoder(174, 128, k_reg=tf.keras.regularizers.L2(), act_reg=SparseRegularizer(), adjacency=t1_A)

In [9]:
history_t1 = train(model_t1, 1000, 174, t1_B)

In [11]:
H = model_t1.encoder(t1_B)
labels = cora_labels(t1)
kmeans = KMeans(n_clusters=5, n_init=20).fit(H)

normalized_mutual_info_score(labels, kmeans.labels_)

0.1191718601979415

In [13]:
H_1 = model_t1.encoder(t1_B)
model_t1_2 = Autoencoder(128, 64, k_reg=tf.keras.regularizers.L2(), act_reg=SparseRegularizer())
history_t1_2 = train(model_t1_2, 1000, 174, H_1)

In [46]:
test_history_3

{0: <tf.Tensor: shape=(), dtype=float32, numpy=0.09875856>,
 1: <tf.Tensor: shape=(), dtype=float32, numpy=0.01083891>,
 2: <tf.Tensor: shape=(), dtype=float32, numpy=0.00067928236>,
 3: <tf.Tensor: shape=(), dtype=float32, numpy=5.9363843e-05>,
 4: <tf.Tensor: shape=(), dtype=float32, numpy=1.4330235e-05>,
 5: <tf.Tensor: shape=(), dtype=float32, numpy=5.541607e-06>,
 6: <tf.Tensor: shape=(), dtype=float32, numpy=2.6446141e-06>,
 7: <tf.Tensor: shape=(), dtype=float32, numpy=1.4164217e-06>,
 8: <tf.Tensor: shape=(), dtype=float32, numpy=8.1478476e-07>,
 9: <tf.Tensor: shape=(), dtype=float32, numpy=4.920088e-07>,
 10: <tf.Tensor: shape=(), dtype=float32, numpy=3.075973e-07>,
 11: <tf.Tensor: shape=(), dtype=float32, numpy=1.9712854e-07>,
 12: <tf.Tensor: shape=(), dtype=float32, numpy=1.2903682e-07>,
 13: <tf.Tensor: shape=(), dtype=float32, numpy=8.5602935e-08>,
 14: <tf.Tensor: shape=(), dtype=float32, numpy=5.7633642e-08>,
 15: <tf.Tensor: shape=(), dtype=float32, numpy=3.9197285e-

In [6]:
t1_S

<tf.Tensor: shape=(174, 174), dtype=float32, numpy=
array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  0., ..., nan, nan, nan],
       [ 0.,  0.,  0., ..., nan, nan, nan]], dtype=float32)>